<a href="https://colab.research.google.com/github/lqh52/LicencePlateRecognition/blob/main/Character_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import numpy as np
import glob
import cv2
from google.colab.patches import cv2_imshow

from keras import optimizers
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [ ]:
ALPHA_DICT = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'K', 9: 'L', 10: 'M', 11: 'N', 12: 'P',
              13: 'R', 14: 'S', 15: 'T', 16: 'U', 17: 'V', 18: 'X', 19: 'Y', 20: 'Z', 21: '0', 22: '1', 23: '2', 24: '3',
              25: '4', 26: '5', 27: '6', 28: '7', 29: '8', 30: '9'}

In [ ]:
data = []
for key in ALPHA_DICT:
  label = key
  img_path = '/content/drive/MyDrive/LicensePlateDetection/data_CNN/'+ALPHA_DICT[key]+'/*.jpg'
  img_list = glob.glob(img_path)
  for img in img_list:
    img_data = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    img_data = cv2.resize(img_data, (28, 28), cv2.INTER_AREA)
    img_data = img_data.reshape((28, 28, 1))
    data.append((img_data, label))

In [ ]:
np.random.shuffle(data)
images = []
labels = []

for i in range(len(data)):
  image, label = data[i]
  images.append(image)
  labels.append(label)

labels = to_categorical(labels, num_classes=31)

In [ ]:
BATCH_SIZE = 128
EPOCHS = 50

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(31, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(1e-3), metrics=['acc'])

In [ ]:
batch_size = BATCH_SIZE
num_epochs = EPOCHS

In [ ]:
# reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, )
# Model Checkpoint
cpt_save = ModelCheckpoint('./weight.h5', save_best_only=True, monitor='val_acc', mode='max')
trainX, trainY = images, labels
N = len(trainX)
valX = trainX[:int(N*0.2)]
valY = trainY[:int(N*0.2)]
trainX = trainX[int(N*0.2):]
trainY = trainY[int(N*0.2):]
trainX = np.array(trainX)
valX = np.array(valX)
model.fit(trainX, trainY, callbacks=[cpt_save, reduce_lr], verbose=1, epochs=num_epochs, shuffle=True, batch_size=batch_size, validation_data=(valX, valY))

Epoch 1/50
23/23 [==============================] - 1s 51ms/step - loss: 2.6949 - acc: 0.2370 - val_loss: 1.8384 - val_acc: 0.4443
Epoch 2/50
23/23 [==============================] - 0s 12ms/step - loss: 1.7683 - acc: 0.4548 - val_loss: 0.9226 - val_acc: 0.6713
Epoch 3/50
23/23 [==============================] - 0s 11ms/step - loss: 1.1947 - acc: 0.6190 - val_loss: 0.5518 - val_acc: 0.8404
Epoch 4/50
23/23 [==============================] - 0s 12ms/step - loss: 0.8754 - acc: 0.6946 - val_loss: 0.4240 - val_acc: 0.8542
Epoch 5/50
23/23 [==============================] - 0s 14ms/step - loss: 0.7257 - acc: 0.7537 - val_loss: 0.3257 - val_acc: 0.8927
Epoch 6/50
23/23 [==============================] - 0s 12ms/step - loss: 0.6146 - acc: 0.7946 - val_loss: 0.3022 - val_acc: 0.8776
Epoch 7/50
23/23 [==============================] - 0s 14ms/step - loss: 0.5386 - acc: 0.8148 - val_loss: 0.2433 - val_acc: 0.9216
Epoch 8/50
23/23 [==============================] - 0s 12ms/step - loss: 0.4661 - a